In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F



In [2]:
# print(torch.__version__)

import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

g = dataset[0]


  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [7]:
print(g)

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'feat': Scheme(shape=(1433,), dtype=torch.float32)}
      edata_schemes={'__orig__': Scheme(shape=(), dtype=torch.int64)})


In [9]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)


In [11]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

g = g.to('cuda')
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes).to('cuda')
train(g, model)

In epoch 0, loss: 1.946, val acc: 0.100 (best 0.100), test acc: 0.111 (best 0.111)
In epoch 5, loss: 1.886, val acc: 0.338 (best 0.338), test acc: 0.384 (best 0.384)
In epoch 10, loss: 1.802, val acc: 0.534 (best 0.534), test acc: 0.585 (best 0.585)
In epoch 15, loss: 1.695, val acc: 0.590 (best 0.590), test acc: 0.629 (best 0.629)
In epoch 20, loss: 1.567, val acc: 0.632 (best 0.632), test acc: 0.657 (best 0.657)
In epoch 25, loss: 1.420, val acc: 0.664 (best 0.664), test acc: 0.677 (best 0.677)
In epoch 30, loss: 1.259, val acc: 0.698 (best 0.698), test acc: 0.697 (best 0.697)
In epoch 35, loss: 1.091, val acc: 0.712 (best 0.712), test acc: 0.716 (best 0.716)
In epoch 40, loss: 0.924, val acc: 0.722 (best 0.722), test acc: 0.730 (best 0.730)
In epoch 45, loss: 0.767, val acc: 0.738 (best 0.738), test acc: 0.744 (best 0.741)
In epoch 50, loss: 0.627, val acc: 0.746 (best 0.748), test acc: 0.754 (best 0.751)
In epoch 55, loss: 0.507, val acc: 0.750 (best 0.750), test acc: 0.754 (best 0